# Scheduled Integration of ClinVar Gene Variant-Disease Data into WikiData

ClinVar aggregates information about genomic variation and its relationship to human health <br>
CC0 https://www.ncbi.nlm.nih.gov/clinvar/

This scheduled bot operates monthly through WDI to integrate ClinVar Gene Variant-Disease Data <br>
https://www.ncbi.nlm.nih.gov/clinvar/docs/ftp_primer/ (variant_summary) <br>
https://github.com/SuLab/GeneWikiCentral/issues/50 <br>
http://jenkins.sulab.org/ <br>

Python script contributions, in order: Sabah Ul-Hasan, Andrew I Su, Tong Shu Li

## Checks and Tests

- x

In [33]:
# Relevant Modules and Libraries

import os # OS package to ensure interaction between the modules (ie WDI) and current OS being used

from datetime import datetime # For identifying the current date and time
import time # Keep track of total for loop run time

import gzip # For unzip of files
import shutil # Copies content of source file(s)
import csv # For converting file(s) to csv format

import pandas as pd # For data organization, abbreviated to pd

from wikidataintegrator import wdi_core, wdi_login # Core and login from wikidataintegrator module
from wikidataintegrator.ref_handlers import update_retrieved_if_new_multiple_refs # For retrieving references

In [58]:
# Download data from NCBI

## Make sure os has wget installed, or the following command wont work
os.system('wget ftp://ftp.ncbi.nlm.nih.gov/pub/clinvar/tab_delimited/variant_summary.txt.gz') 
timeStringNow = datetime.now().strftime("+%Y-%m-%dT00:00:00Z") # time stamp of download 

## Unzip the file
with gzip.open('variant_summary.txt.gz', 'rb') as f_in:
    with open('variant_summary.txt', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
        
## Convert .txt file to .csv
txt_file = r"variant_summary.txt"
csv_file = r"variant_summary.csv"

with open(txt_file, "r") as in_text:
    in_reader = csv.reader(in_text, delimiter = '\t')
    with open(csv_file, "w") as out_csv:
        out_writer = csv.writer(out_csv)
        for row in in_reader:
            out_writer.writerow(row)

## Import .csv file and read first 5 rows
df = pd.read_csv("variant_summary.csv") 
df.shape # 31 columns, 1330018 rows
df.head()

,#AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,LastEvaluated,RS# (dbSNP),...,ReferenceAllele,AlternateAllele,Cytogenetic,ReviewStatus,NumberSubmitters,Guidelines,TestedInGTR,OtherIDs,SubmitterCategories,VariationID
0,15041,indel,NM_014855.3(AP5Z1):c.80_83delinsTGCTGTAAACTGTA...,9907,AP5Z1,HGNC:22197,Pathogenic,1,"Jun 29, 2010",397704705,...,GGAT,TGCTGTAAACTGTAACTGTAAA,7p22.1,no assertion criteria provided,1,NaN,N,OMIM Allelic Variant:613653.0001,1,2
1,15041,indel,NM_014855.3(AP5Z1):c.80_83delinsTGCTGTAAACTGTA...,9907,AP5Z1,HGNC:22197,Pathogenic,1,"Jun 29, 2010",397704705,...,GGAT,TGCTGTAAACTGTAACTGTAAA,7p22.1,no assertion criteria provided,1,NaN,N,OMIM Allelic Variant:613653.0001,1,2
2,15042,deletion,NM_014855.3(AP5Z1):c.1413_1426del (p.Leu473fs),9907,AP5Z1,HGNC:22197,Pathogenic,1,"Jun 29, 2010",397704709,...,GCTGCTGGACCTGCC,G,7p22.1,no assertion criteria provided,1,NaN,N,OMIM Allelic Variant:613653.0002,1,3
3,15042,deletion,NM_014855.3(AP5Z1):c.1413_1426del (p.Leu473fs),9907,AP5Z1,HGNC:22197,Pathogenic,1,"Jun 29, 2010",397704709,...,GCTGCTGGACCTGCC,G,7p22.1,no assertion criteria provided,1,NaN,N,OMIM Allelic Variant:613653.0002,1,3
4,15043,single nucleotide variant,NM_014630.3(ZNF592):c.3136G>A (p.Gly1046Arg),9640,ZNF592,HGNC:28986,Uncertain significance,0,"Jun 29, 2015",150829393,...,G,A,15q25,no assertion criteria provided,1,NaN,N,"OMIM Allelic Variant:613624.0001,UniProtKB (pr...",1,4


In [59]:
# Clean-up of data for identification and integration

## Columns to keep
new = df[['Type', 'Name', 'GeneSymbol','HGNC_ID', 'ClinicalSignificance', 'PhenotypeIDS', 'PhenotypeList','VariationID', 
            'ChromosomeAccession', 'Chromosome', 'Start', 'ReferenceAllele', 'AlternateAllele',
            'ReviewStatus']]

## Create new column that converts 'ReviewStatus' to star rating
### https://www.ncbi.nlm.nih.gov/clinvar/docs/review_status/
new['Rating'] = "" # Create empty column for gold star rating
## Convert strings from 'ReviewStatus' to 'Rating' 
new.loc[new['ReviewStatus'].str.contains('no assertion provided'), 'Rating'] = 'none'
new.loc[new['ReviewStatus'].str.contains('no assertion criteria provided'), 'Rating'] = 'none'
new.loc[new['ReviewStatus'].str.contains('no assertion for the individual variant'), 'Rating'] = 'none'
new.loc[new['ReviewStatus'].str.contains('criteria provided, single submitter'), 'Rating'] = 'one'
new.loc[new['ReviewStatus'].str.contains('criteria provided, conflicting interpretations'), 'Rating'] = 'one'
new.loc[new['ReviewStatus'].str.contains('criteria provided, conflicting interpretations'), 'Rating'] = 'one'
new.loc[new['ReviewStatus'].str.contains('criteria provided, multiple submitters, no conflicts'), 'Rating'] = 'two'
new.loc[new['ReviewStatus'].str.contains('reviewed by expert panel'), 'Rating'] = 'three'
new.loc[new['ReviewStatus'].str.contains('practice guideline'), 'Rating'] = 'four'

new.shape # 15 columns, 1330018 rows 

## Training dataset, based on the following criteria
threeplus=new[new['Rating'].str.contains('three|four')]
snv=threeplus[threeplus['Type'].str.contains('single nucleotide variant')]
patho=snv[snv['ClinicalSignificance'].str.contains('Pathogenic')]

patho.shape # 15 columns, 3870 rows (0.29% of original dataset)

## Create HGVS columns and IDs (based on genomic position)
### Naming nomenclature: https://varnomen.hgvs.org/bg-material/numbering/
patho['HGVS_NC'] = "" # Create empty column for HGVS nomenclature with NC
patho['HGVS_chr'] = "" # Create empty column for HGVS nomenclature with chr

patho['HGVS_NC']=patho['ChromosomeAccession']+':g.'+patho['Start'].astype(str)+patho['ReferenceAllele']+'>'+patho['AlternateAllele']
patho['HGVS_chr']='chr'+patho['Chromosome'].astype(str)+':g.'+patho['Start'].astype(str)+patho['ReferenceAllele']+'>'+patho['AlternateAllele']

df=patho
df.head()

/Users/sulhasan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/sulhasan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/sulhasan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

,Type,Name,GeneSymbol,HGNC_ID,ClinicalSignificance,PhenotypeIDS,PhenotypeList,VariationID,ChromosomeAccession,Chromosome,Start,ReferenceAllele,AlternateAllele,ReviewStatus,Rating,HGVS_NC,HGVS_chr
1088,single nucleotide variant,NM_000277.3(PAH):c.1315+1G>A,PAH,HGNC:8582,Pathogenic,"MeSH:D030342,MedGen:C0950123;MedGen:C0751434,O...",Inborn genetic diseases;Phenylketonuria;not pr...,576,NC_000012.11,12,103234177,C,T,reviewed by expert panel,three,NC_000012.11:g.103234177C>T,chr12:g.103234177C>T
1089,single nucleotide variant,NM_000277.3(PAH):c.1315+1G>A,PAH,HGNC:8582,Pathogenic,"MeSH:D030342,MedGen:C0950123;MedGen:C0751434,O...",Inborn genetic diseases;Phenylketonuria;not pr...,576,NC_000012.12,12,102840399,C,T,reviewed by expert panel,three,NC_000012.12:g.102840399C>T,chr12:g.102840399C>T
1090,single nucleotide variant,NM_000277.3(PAH):c.1222C>T (p.Arg408Trp),PAH,HGNC:8582,Pathogenic,"MedGen:C0751434,OMIM:261600,Orphanet:ORPHA716,...",Phenylketonuria;not provided,577,NC_000012.11,12,103234271,G,A,reviewed by expert panel,three,NC_000012.11:g.103234271G>A,chr12:g.103234271G>A
1091,single nucleotide variant,NM_000277.3(PAH):c.1222C>T (p.Arg408Trp),PAH,HGNC:8582,Pathogenic,"MedGen:C0751434,OMIM:261600,Orphanet:ORPHA716,...",Phenylketonuria;not provided,577,NC_000012.12,12,102840493,G,A,reviewed by expert panel,three,NC_000012.12:g.102840493G>A,chr12:g.102840493G>A
1098,single nucleotide variant,NM_000277.3(PAH):c.331C>T (p.Arg111Ter),PAH,HGNC:8582,Pathogenic,"MedGen:C0751434,OMIM:261600,Orphanet:ORPHA716,...",Phenylketonuria;not provided,581,NC_000012.11,12,103288534,G,A,reviewed by expert panel,three,NC_000012.11:g.103288534G>A,chr12:g.103288534G>A


In [34]:
# Login for running WDI
print("Logging in...") 

# **remove lines when scheduling to Jenkins** Enter your own username and password 
os.environ["WDUSER"] = "username" # Uses os package to call and set the environment for wikidata username
os.environ["WDPASS"] = "password"

## Conditional that outputs error command if not in the local python environment
if "WDUSER" in os.environ and "WDPASS" in os.environ: 
    WDUSER = os.environ['WDUSER']
    WDPASS = os.environ['WDPASS']
else: 
    raise ValueError("WDUSER and WDPASS must be specified in local.py or as environment variables")      

## Sets attributed username and password as 'login'
login = wdi_login.WDLogin(WDUSER, WDPASS) 

Logging in...
https://www.wikidata.org/w/api.php
Successfully logged in as Sulhasan


In [65]:
df = df[(df['HGVS_NC'].str.contains("NC_000012.11:g.103234177C>T|NC_000002.11:g.47656951C>T|NC_000003.11:g.37038192G>A|NC_000003.11:g.37042536C>T|NC_000003.11:g.37045935C>T|NC_000003.11:g.37048546C>T|NC_000003.11:g.37053589C>T|NC_000003.11:g.37056036G>A|NC_000017.10:g.7577539G>A|NC_000017.10:g.7577548C>T|NC_000017.10:g.7578190T>C|NC_000021.8:g.36171704G>T|NC_000021.8:g.36252962C>G|NC_000021.8:g.36259163T>C|NC_000010.10:g.89711899C>T|NC_000012.11:g.103310908T>C|NC_000012.12:g.102917130T>C|NC_000017.10:g.7577120C>T|NC_000017.10:g.7577538C>T"))|(df['HGVS_chr'].str.contains("chr17:g.41228590G>A|chr17:g.41234451G>A"))]
df.shape

# First 1 NC does not match to Wikidata, chr (2) do not match to Wikidata *18 possible writes

(19, 17)

In [66]:
# For loop on training dataset 
start_time = time.time() # Keep track of how long it takes loop to run

for index, row in df.iterrows(): # Index is a row number, row is all variables and values for that row
    
    # Identify string for given row to retrive variant name and item page Qid from Wikidata
    HGVS_NC = df.loc[index, 'HGVS_NC']
    HGVS_chr = df.loc[index, 'HGVS_chr']
    
    # SparQL query to search HGVS Identifier in Wikidata based on P3331 
    sparqlQuery_HGVSNC = "SELECT * WHERE {?variant wdt:P3331 \""+HGVS_NC+"\"}" 
    result_HGVSNC = wdi_core.WDItemEngine.execute_sparql_query(sparqlQuery_HGVSNC) 
    sparqlQuery_HGVSchr = "SELECT * WHERE {?variant wdt:P3331 \""+HGVS_chr+"\"}" 
    result_HGVSchr = wdi_core.WDItemEngine.execute_sparql_query(sparqlQuery_HGVSchr)   
    
    # Assign a length to the resultant dictionary for either NC or chr (number of Qid)
    NC_qlength = len(result_HGVSNC["results"]["bindings"]) 
    chr_qlength = len(result_HGVSchr["results"]["bindings"]) 
    
    # Match HGVS NC* or chr* created for ClinVar dataset to Wikidata Qid item page
    if NC_qlength == 0 & chr_qlength == 0:
        print("Need item page") # Write item page here, log (assumption no item page exists) *add HGVS
        continue
        
    if NC_qlength > 1:
        print("error: multiple qid for nc") # log
        if chr_qlength > 1:
            print("error: multiple qid for chr") # log
            continue
    
    if chr_qlength > 1:
        print("error: multiple qid for chr") # log
        continue
        
    if NC_qlength == 1:
        NC_qid = result_HGVSNC["results"]["bindings"][0]["variant"]["value"].replace("http://www.wikidata.org/entity/", "")
        print("NC Qid", NC_qid)
        # Add ClinVar Variation ID (stated in, ref, retrieved)
        if chr_qlength == 0:
            print("Need chr in HGVS") # Write identifier, keep working through loop
        if chr_qlength == 1:
            chr_qid = result_HGVSchr["results"]["bindings"][0]["variant"]["value"].replace("http://www.wikidata.org/entity/", "")
            if NC_qid == chr_qid:
                print("same qid") # Keep working through loop
                # look at diseases (add genetic association)
                # Symmetry for diseases
                
# Separate out Phenotype List (;) for disease 
    # Split one line into two (ie if there are two diseases)
        # Split by commas for identifiers
            # For example, if there are four identifiers
            # Take all identifiers for that disease, and search against Wikidata
            # Take union of results
                # If only one QiD found, then good
                # If there are multiple, then flag (don't write anything) *dont write anything for whole line
                # Add identifiers from dataset that are missing in Wikidata
                
            else:
                print("error: different qid")
                continue
    
    if chr_qlength == 1:
        chr_qid = result_HGVSchr["results"]["bindings"][0]["variant"]["value"].replace("http://www.wikidata.org/entity/", "")
        print("chr Qid", chr_qid)
        print("Need NC in HGVS") # Add genetic association, assumption NC_qlength is 0 based on above
        # Add ClinVar Variation ID (stated in, ref, retrieved)
        
# NC, chr (HGVS identifiers)
# 0, 0 = absent => write item page
# 1, 0 = result => continue
# 0, 1 = result => continue
# 1, 1 = same => continue
# 1, 1 = different => error
        
end_time = time.time() # Captures when loop run ends
print("The total time of this loop is:", end_time - start_time, "seconds, or", (end_time - start_time)/60, "minutes")

Need item page
NC Qid Q64401263
Need chr in HGVS
NC Qid Q64401263
Need chr in HGVS
NC Qid Q28371040
Need chr in HGVS
NC Qid Q28371039
Need chr in HGVS
NC Qid Q29938325
Need chr in HGVS
NC Qid Q28371044
Need chr in HGVS
NC Qid Q28371675
Need chr in HGVS
NC Qid Q28371676
Need chr in HGVS
NC Qid Q28371680
Need chr in HGVS
NC Qid Q28599642
Need chr in HGVS
NC Qid Q28599631
Need chr in HGVS
NC Qid Q28599630
Need chr in HGVS
NC Qid Q28599645
Need chr in HGVS
NC Qid Q28599641
Need chr in HGVS
NC Qid Q28599625
Need chr in HGVS
NC Qid Q28599622
Need chr in HGVS
NC Qid Q29938384
Need chr in HGVS
NC Qid Q29938054
Need chr in HGVS
The total time of this loop is: 9.418941974639893 seconds, or 0.15698236624399822 minutes


In [67]:
## Download HGVS Wikidata query results as query.csv 
### https://query.wikidata.org/#SELECT%20%2a%20WHERE%20%7B%3Fgene%20wdt%3AP3331%20%3FHGVS%7D

query = pd.read_csv("/Users/sulhasan/Desktop/Su Lab Projects/ClinVar-Bot_GeneWikiCentral-Issue50/query.csv")  

## Subset for NC or chr
query = query[query['HGVS'].str.contains('chr|NC_',  na=False)] # 818 with NC or chr
query.head(5) 

# How many HGVS IDs match for Wikidata query (818) vs. manually created in ClinVar (3778 x 2 for NC or chr)
len(set(query["HGVS"]) & (set(df["HGVS_chr"]) | set(df["HGVS_NC"]))) # 17
set(query["HGVS"]) & (set(df["HGVS_chr"]) | set(hasHGVS["HGVS_NC"])) # 4 chr, 13 NC *chr dont match...

{'NC_000002.11:g.47656951C>T',
 'NC_000003.11:g.37038192G>A',
 'NC_000003.11:g.37042536C>T',
 'NC_000003.11:g.37045935C>T',
 'NC_000003.11:g.37048546C>T',
 'NC_000003.11:g.37053589C>T',
 'NC_000003.11:g.37056036G>A',
 'NC_000010.10:g.89711899C>T',
 'NC_000012.11:g.103310908T>C',
 'NC_000012.12:g.102917130T>C',
 'NC_000017.10:g.7577120C>T',
 'NC_000017.10:g.7577538C>T',
 'NC_000017.10:g.7577539G>A',
 'NC_000017.10:g.7577548C>T',
 'NC_000017.10:g.7578190T>C',
 'NC_000021.8:g.36171704G>T',
 'NC_000021.8:g.36252962C>G',
 'NC_000021.8:g.36259163T>C'}

In [ ]:
# Keep anything with 'two' or more stars in 'Rating' column
twoplus = new[~new['Rating'].str.contains('one')] # excludes both one and none
twoplus.shape # 13 columns, 196530 rows: 1123285 removed, 14.9 % of all data usable
# Keep anything noted as 'single nucleotide variant' in 'Type' column
snv = twoplus[twoplus['Type'].str.contains('single nucleotide variant')]
snv.shape # 174733 rows, or 88.9% of data with two or more stars (13.2% of all data)
## 83.2 % of total data, prior to star rating filter, are SNVs
# Keep anything with 'Pathogenic' in the 'ClinicalSignificance' column
patho = snv[snv['ClinicalSignificance'].str.contains('Pathogenic')]
patho.shape # 20040 rows, or 10.2% of data with two or more stars that are snvs (1.5% of all data)
## 12.8 % of total data, prior to star rating filter, are Pathogenic